In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz
!pip install textacy
!pip install negspacy
!python -m spacy validate
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578645 sha256=d8bbc865dcaf1ff2eef04e7ca354858d0eac007b289c6524f1a83eee0de13b50
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [ ]:
import nltk
import spacy
from google.colab import drive
import en_core_web_sm
import spacy
from negspacy.negation import Negex
import pandas as pd
import numpy as np
import csv
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#基本詞性分析
def doclen(line1):
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
  return len(doc)



In [ ]:
#詞分類
def words_recognize(line1):
  word_dic={}
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  pos_tag=['SPACE','ADV','VERB','ADP','DET','NOUN','ADJ','PUNCT','INTJ','NUM','PRON','AUX','CCONJ','PART','PROPN','SCONJ','X','CONJ','SYM']
  for tag in pos_tag:
    list1=[]
    word_dic.update({tag:list1})
  for token in doc:
    word_dic[token.pos_].append(token.text)
  return word_dic

In [ ]:
#counting lemma
def counting_lemma(line1):
  lemma_dict={}
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  for token in doc:
    if(token.lemma_ not in lemma_dict):
      lemma_dict.update({token.lemma_:1})
    else:
      lemma_dict[token.lemma_]+=1
  #for lemma in lemma_dict:
    #print(lemma,lemma_dict[lemma])
  sort_dict=dict(sorted(lemma_dict.items(), key=lambda item: item[1]))
  max_freq=-1
  most_freq_words="none"
  for item in sort_dict:
    if(sort_dict[item]>max_freq and item!='.' or item!=','):
      max_freq=sort_dict[item]
      most_freq_words=item
  return len(lemma_dict),max_freq

In [ ]:
#noun chunks
def counting_numnchunk(line1):
  num=0
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  for chunk in doc.noun_chunks:
      num+=1
  return num

In [ ]:
#first person singular verbs
def first_person_singular_verbs(line1):
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  singular_verbs=0
  for i in range(0,len(doc)):
    if(doc[i].pos_=="VERB"):
      if(i>=1 and doc[i-1].text=="I"):
        singular_verbs+=1
  return singular_verbs


In [ ]:
#誤拼字分析
def mis_spell(line1):
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  propn=[]
  propnum=0
  x_word=0
  for token in doc:
    if(token.pos_=="PROPN"):
      propnum+=1
      propn.append(token.text)
  for token in doc:
    if(token.pos_=="X" or token.pos_=="SYM"):
      x_word+=1
  real_propn=0
  for ent in doc.ents:
    if(ent.text in propn):
      real_propn+=1
  return propnum+x_word-real_propn

In [ ]:
#Absolute time term counting
def NES(line1):
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  time_term=0
  term=0
  for ent in doc.ents:
    if(ent.label_=="TIME"):
      time_term+=1
    term+=1
  return time_term,term

In [ ]:
from spacy.matcher import PhraseMatcher
from negspacy.termsets import termset
from negspacy.negation import Negex

In [ ]:
#否定詞找尋v2(final version)
def neg_analyze(line1):
  nlp = spacy.load("en_core_web_sm")
  nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})
  doc = nlp(line1)
  negnum=0
  for e in doc.ents:
    negnum+=1
  return negnum

In [ ]:
#虛實詞
def content_function(line1):
  nlp = en_core_web_sm.load()
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(line1)
  content_num=0
  content_exception=["no", "not", "never", "this", "that", "these" ,"those","what", "where", "when", "how" , "why"]
  for token in doc:
    if(token.pos_=="NOUN"or token.pos_=="VERB" or token.pos_=="ADJ" or token.pos_=="ADV"):
      content_num+=1
    else:
      if(token.text in content_exception):
        content_num+=1
  function_word=0
  for token in doc:
    if(token.pos_=="AUX" or token.pos_=="ADP" or token.pos_=="DET" or token.pos_=="CONJ" or token.pos_=="CCONJ" or token.pos_=="SCONJ" or token.pos_=="PRON"):
      if(token.text not in content_exception):
        function_word+=1
  return content_num,function_word


In [ ]:
prev_row=[]
with open('/content/drive/MyDrive/term project/ted_npl_result.csv', 'r', newline='') as csvfile:
    # Create a CSV reader object
    csv_reader = csv.reader(csvfile)
    # Read each row in the CSV file
    for i ,row in  zip(range(0,504),csv_reader):
        prev_row.append(row)
    print(len(prev_row))


504


In [ ]:
filenames = []
# Read in file names that are needed to convert
f = open('/content/drive/MyDrive/Colab Notebooks/term project/ted_info.txt')
for line in f.readlines():
    filenames.append(line[0:len(line)-1])
f.close

print(filenames)
feature_table=np.zeros((len(filenames),57))

# Convert transcritps into features
for i in range(filenames.index("NateSilver_2009.sph")+1,(len(filenames))):
  print(filenames[i])
  try:
    source_file_path='/content/drive/MyDrive/Colab Notebooks/term project/TED text/'+filenames[i][0:len(filenames[i])-4]+'.txt'
    f1 = open(source_file_path, "r")
    line1 = f1.readline()
    f1.close()
    words=doclen(line1)
    feature_table[i][0]=doclen(line1)
    word_dict=words_recognize(line1)
    feature_table[i][1]=len(word_dict["SPACE"])
    feature_table[i][2]=len(word_dict["ADV"])
    feature_table[i][3]=len(word_dict["VERB"])
    feature_table[i][4]=len(word_dict["ADP"])
    feature_table[i][5]=len(word_dict["DET"])
    feature_table[i][6]=len(word_dict["NOUN"])
    feature_table[i][7]=len(word_dict["ADJ"])
    feature_table[i][8]=len(word_dict["PUNCT"])
    feature_table[i][9]=len(word_dict["INTJ"])
    feature_table[i][10]=len(word_dict["NUM"])
    feature_table[i][11]=len(word_dict["PRON"])
    feature_table[i][12]=len(word_dict["AUX"])
    feature_table[i][13]=len(word_dict["CCONJ"])
    feature_table[i][14]=len(word_dict["PART"])
    feature_table[i][15]=len(word_dict["PROPN"])
    feature_table[i][16]=len(word_dict["SCONJ"])
    feature_table[i][17]=len(word_dict["CONJ"])
    feature_table[i][18]=len(word_dict["PUNCT"])
    feature_table[i][19]=len(word_dict["INTJ"])
    feature_table[i][20],feature_table[i][21]=counting_lemma(line1)
    feature_table[i][22]=counting_numnchunk(line1)
    feature_table[i][23]=first_person_singular_verbs(line1)
    feature_table[i][24]=mis_spell(line1)
    feature_table[i][25],feature_table[i][26]=NES(line1)
    feature_table[i][27]=len(line1.split('.'))
    feature_table[i][28]=neg_analyze(line1)
    feature_table[i][29],feature_table[i][30]=content_function(line1)
    feature_table[i][31]=len(word_dict["SPACE"])/words
    feature_table[i][32]=len(word_dict["ADV"])/words
    feature_table[i][33]=len(word_dict["VERB"])/words
    feature_table[i][34]=len(word_dict["ADP"])/words
    feature_table[i][35]=len(word_dict["DET"])/words
    feature_table[i][36]=len(word_dict["NOUN"])/words
    feature_table[i][37]=len(word_dict["ADJ"])/words
    feature_table[i][38]=len(word_dict["PUNCT"])/words
    feature_table[i][39]=len(word_dict["INTJ"])/words
    feature_table[i][40]=len(word_dict["NUM"])/words
    feature_table[i][41]=len(word_dict["PRON"])/words
    feature_table[i][42]=len(word_dict["AUX"])/words
    feature_table[i][43]=len(word_dict["CCONJ"])/words
    feature_table[i][44]=len(word_dict["PART"])/words
    feature_table[i][45]=len(word_dict["PROPN"])/words
    feature_table[i][46]=len(word_dict["SCONJ"])/words
    feature_table[i][47]=len(word_dict["CONJ"])/words
    feature_table[i][48]=len(word_dict["PUNCT"])/words
    feature_table[i][49]=len(word_dict["INTJ"])/words
    feature_table[i][50],x=counting_lemma(line1)#20
    feature_table[i][50]/=words
    feature_table[i][51]=first_person_singular_verbs(line1)/words#23
    feature_table[i][52]=mis_spell(line1)/words#24
    feature_table[i][53],feature_table[i][54]=NES(line1)#25 26
    feature_table[i][53]/=words
    feature_table[i][54]/=words
    feature_table[i][54]=neg_analyze(line1)/words#28
    feature_table[i][55],feature_table[i][56]=content_function(line1)#29 30
    feature_table[i][55]/=words
    feature_table[i][56]/=words
  except:
    continue

FileNotFoundError: ignored

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/term project/ted_npl_result.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['name', 'words_number', 'SPACE','ADV','VERB','ADP','DET','NOUN','ADJ','PUNCT','INTJ','NUM','PRON','AUX','CCONJ','PART','PROPN','SCONJ','CONJ','Punctuation','hestitation_word','lemma_number','most_frequent','noun_chunk','person_singular_verbs','misspell','time_spec','spec','sentence','neg_word','content','function','SPACE_R','ADV_R','VERB_R','ADP_R','DET_R','NOUN_R','ADJ_R','PUNCT_R','INTJ_R','NUM_R','PRON_R','AUX_R','CCONJ_R','PART_R','PROPN_R','SCONJ_R','CONJ_R','Punctuation_R','hestitation_word_R','lemma_number_R','person_singular_verbs_R','misspell_R','time_spec_R','spec_R','neg_word_R','content_R','function_R'])
  for i in range(1,len(prev_row)):
    writer.writerow(prev_row[i])
  for i in range(filenames.index("NateSilver_2009.sph")+1,len(filenames)):
    row=[]
    row.append(filenames[i])
    for j in range(0,57):
      row.append(feature_table[i][j])
    writer.writerow(row)